In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
import torch
print(torch.__version__)


2.0.1+cu117


In [3]:
print(torch.version.cuda)

11.7


In [4]:
import torch
torch.ones((1,)).cuda()

tensor([1.], device='cuda:0')

In [5]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

True
1
NVIDIA RTX A4500
